PWC Stock price sentiment analysis using Twitter API
----------

Sentiment in the social media (facebook, twitter, instagram, linkedin etc.) plays a big role in managing the perception of an organisation. This sentiment can have an impact in the stock price of listed companies. Is there a statistically significant correlation (or even a lagged correlation) between the social media sentiment and the stock price. This will give the corporate communications department a very important tool to monitor the leading indicator (social media sentiment) in order to estimate the lagging indicator (stock price)

In [2]:
# Imports
import re
import tweepy
from tweepy import OAuthHandler
from textblob import TextBlob

In [3]:
# Twitter Client Class    
class TwitterClient(object):
    
    def __init__(self):
        # keys and tokens from the Twitter Dev Console
        consumer_key = 'Ve8tKh7aP2CtGhFUtV26W7Zr8'
        consumer_secret = 'P7L9hsifW5cRT4Yg9eJHzDtmi1RZ42ENSWwcKLp7SkegoG4TYT'
        access_token = '2918675084-KoiDatN9UJnvOKJxv4E2eFXmbh94GTiydawSf9w'
        access_token_secret = 'xpcA5NEFyGZ6WQZ5leXpBqHJDS8DlBBseQGqJpJQJU6Y9'
 
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
 
    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())
 
    def get_tweet_sentiment(self, tweet):
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
 
    def get_tweets(self, query, count = 10):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
 
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)
 
            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
 
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
 
                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
 
            # return parsed tweets
            return tweets
 
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))

In [4]:
# Main function
def main():
    # creating object of TwitterClient Class
    api = TwitterClient()
    # calling function to get tweets
    query = input('Enter the stock name listed in BSE or NSE : ')
    epoch = int(input('Enter the number of tweets to be taken : '))
    tweets = api.get_tweets(query = query, count = epoch)
 
    # picking positive tweets from tweets
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    # percentage of positive tweets
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
    # picking negative tweets from tweets
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    # percentage of negative tweets
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
    # percentage of neutral tweets
    # print("Neutral tweets percentage: {} %".format(100*len(list(set(tweets) - set(ntweets) - set(ptweets)))/len(tweets)))
    netweets = [tweet for tweet in tweets if tweet['sentiment'] == 'neutral']
    print("Neutral tweets percentage: {} %".format(100*len(netweets)/len(tweets))) 

    # printing first 5 positive tweets
    print("\n\nPositive tweets:")
    for tweet in ptweets[:10]:
        print(tweet['text'])
 
    # printing first 5 negative tweets
    print("\n\nNegative tweets:")
    for tweet in ntweets[:10]:
        print(tweet['text'])
 

In [5]:
# Calling Main Function
if __name__ == "__main__":
    # calling main function
    main()

Enter the stock name listed in BSE or NSE : Rain Industries
Enter the number of tweets to be taken : 500
Positive tweets percentage: 52.87356321839081 %
Negative tweets percentage: 3.4482758620689653 %
Neutral tweets percentage: 43.67816091954023 %


Positive tweets:
@JAugastine Your suggestions rain industries and graphite india really helped. Thanks. Keep sharing valuable inputs.
Another good article on #RainInd 
https://t.co/Pa2Tq2u6mp
Track Aluminum prices here https://t.co/J5yvM8Laww
RT @Arvind2k: Rain Industries: Good results confirm strong tailwinds for carbon products in metal industry.
currently trading at a multiple…
Rain Industries: Good results confirm strong tailwinds for carbon products in metal industry.
currently trading at… https://t.co/CSdo57SXYB
Strong numbers by Rain Industries
@porinju Would anyone share their thoughts on Rain Industries. Worth getting in now for a 3 year view?? 
I am quite… https://t.co/9OSHOc74Wk
RT @GoldieTuteja: Rain Industries has changed the 